# **CS 363M Final Project Spring 2025**

## Chenyi Wang, Bhuvan Kannaeganti, Suyog Valsangkar

### **Overview**

For the project in this class, you will participate in a machine learning competition where you’ll apply your ML skills to a real-world dataset. You may work individually or in teams of up to 3 students. 

The dataset for this competition comes from the Austin Animal Center, the largest no-kill animal shelter in the United States. It contains historical records of animals that have entered the shelter, including details such as species, breed, age, intake type, medical condition, and other attributes. Each animal in the dataset has a recorded outcome, which represents what eventually happened to the animal after entering the shelter.

Your goal in this competition is to build a machine learning model that predicts the final outcome of each animal admitted to the shelter, based on its intake characteristics. The possible outcomes are:

**- Adopted**: The animal was placed into a new home.<br>
**- Return to Owner**: The animal was reclaimed by its original owner.<br>
**- Euthanasia**: The animal was humanely euthanized due to medical or behavioral concerns.<br>
**- Died**: The animal passed away while in the shelter’s care.<br>
**- Transfer**: The animal was moved to another shelter or rescue organization.<br>

By accurately predicting these outcomes, your model can help identify factors that influence an animal's journey through the shelter system and potentially aid in improving adoption and survival rates, shelter policies, or allocation of resources.


## **Code and Analysis Below:**

1. We need to go through the dataset and examine the existing features for patterns and methods we can feature engineer our data to enhance our final predictions. 

In [1]:
import pandas as pd

animal_data = pd.read_csv('train.csv')
animal_test = pd.read_csv('test.csv')

animal_data.sample(5) # sample some data

,Id,Name,Intake Time,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Outcome Time,Date of Birth,Outcome Type
29424,A824400,NaN,10/13/2020 04:59:00 PM,5710 Jacqueline Lane in Austin (TX),Stray,Injured,Cat,Intact Female,1 month,Domestic Shorthair,Calico,10/16/2020 11:29:00 AM,08/29/2020,Transfer
56853,A753776,*Brixey,07/11/2017 08:15:00 AM,Unknown in Austin (TX),Stray,Normal,Cat,Intact Female,4 months,American Shorthair Mix,Black Smoke,08/19/2017 10:59:00 AM,02/11/2017,Adoption
16584,A680624,Axle,06/04/2014 05:47:00 PM,Fm 620 And Anderson Mill in Austin (TX),Stray,Normal,Dog,Neutered Male,11 years,American Eskimo Mix,White/Red,07/28/2014 01:56:00 PM,07/06/2002,Adoption
94487,A880193,Stella,05/05/2023 03:18:00 PM,16104 Fm 812 in Travis (TX),Stray,Normal,Dog,Intact Female,2 months,German Shepherd,White,05/24/2023 04:18:00 PM,02/18/2023,Adoption
14265,A736950,NaN,10/24/2016 12:37:00 PM,Austin (TX),Owner Surrender,Normal,Cat,Intact Male,4 months,Domestic Shorthair Mix,White/Blue Tabby,10/25/2016 06:04:00 PM,06/09/2016,Adoption


2. **Data Cleaning**: After observing the initial dataset, I see that we need to perform data cleaning. We need to separate the sex upon intake feature as it currently also includes sterilization health status. We also need to fix the age upon intake column as it is using all sorts of time units. The name feature also does not exist within the test set, we need to be careful and only train the model on features that exist within the test set. This also applies to Id, which does not correlate to the Id within the test set. Therefore, we can also eliminate name and id before training the model.

In [2]:
# drop name feature entirely
animal_data.drop("Name", axis=1, inplace=True)

# drop id feature entirely
animal_data.drop("Id", axis=1, inplace=True)

# drop outcome time
animal_data.drop("Outcome Time", axis=1, inplace=True)

# drop birth date
animal_data.drop("Date of Birth", axis=1, inplace=True)

# transform Animal Type into a binary column: Cat -> True, Dog -> False
animal_data['Cat'] = animal_data['Animal Type'].apply(lambda x: True if x == 'Cat' else False)
animal_data.drop('Animal Type', axis=1, inplace=True)

# separate age at intake and reproductive status, create new column
split_cols = animal_data['Sex upon Intake'].str.split(' ', n=1, expand=True)

# handle cases where the split returns "Unknown"
animal_data['Sterilized'] = split_cols[0].map({
    'Neutered': 'True',
    'Spayed': 'True',
    'Intact': 'False',
    'Unknown': 'False'
}).fillna('False')  # unexpected values, let's assume not sterilized

# assign gender, we can keep it a binary by make the feature "Male", where male = true and female = false
animal_data['Male'] = split_cols[1].apply(lambda x: False if x == 'Female' else True)
animal_data.drop("Sex upon Intake", axis=1, inplace=True)

# helper to convert age upon intake to years
def age_to_years(age_str):
    if pd.isna(age_str):
        return None
    
    number, unit = age_str.split()
    number = float(number)
    
    if "year" in unit:
        return number
    elif "month" in unit:
        return number / 12
    elif "week" in unit:
        return number / 52
    elif "day" in unit:
        return number / 365
    else:
        return None  # in case of an unexpected format

# convert age to years
animal_data['Age'] = animal_data['Age upon Intake'].apply(age_to_years)
animal_data.drop('Age upon Intake', axis=1, inplace=True)

pd.set_option('display.max_columns', None)
animal_data.sample(5) # sample some data after transformations

,Intake Time,Found Location,Intake Type,Intake Condition,Breed,Color,Outcome Type,Cat,Sterilized,Male,Age
56432,06/16/2019 01:25:00 PM,2207 Wickersham Lane in Austin (TX),Stray,Sick,Dachshund,Black/Tan,Transfer,False,False,False,0.083333
84594,01/14/2014 04:12:00 PM,12800 Cedar St in Travis (TX),Stray,Normal,Domestic Shorthair Mix,Orange Tabby/White,Adoption,True,False,False,2.000000
86490,12/26/2013 07:28:00 AM,4434 Frontier in Austin (TX),Stray,Injured,Domestic Shorthair Mix,Calico,Adoption,True,True,False,0.583333
106352,03/09/2023 03:34:00 PM,Barwood And Ih 35 in Austin (TX),Stray,Normal,Domestic Medium Hair,Orange Tabby/White,Adoption,True,False,True,1.000000
103526,04/12/2014 01:51:00 PM,Austin (TX),Owner Surrender,Normal,Domestic Shorthair Mix,Torbie,Adoption,True,False,False,1.000000


In [3]:
# drop birth date
animal_test.drop("Date of Birth", axis=1, inplace=True)

# transform Animal Type into a binary column: Cat -> True, Dog -> False
animal_test['Cat'] = animal_test['Animal Type'].apply(lambda x: True if x == 'Cat' else False)
animal_test.drop('Animal Type', axis=1, inplace=True)

# separate age at intake and reproductive status, create new column
split_cols = animal_test['Sex upon Intake'].str.split(' ', n=1, expand=True)

# handle cases where the split returns "Unknown"
animal_test['Sterilized'] = split_cols[0].map({
    'Neutered': True,
    'Spayed': True,
    'Intact': False,
    'Unknown': False
}).fillna(False)  # unexpected values, let's assume not sterilized
animal_data['Sterilized'] = animal_data['Sterilized'].map({'True': True, 'False': False})

# assign gender, we can keep it a binary by make the feature "Male", where male = true and female = false
animal_test['Male'] = split_cols[1].apply(lambda x: False if x == 'Female' else True)
animal_test.drop("Sex upon Intake", axis=1, inplace=True)

# helper to convert age upon intake to years
def age_to_years(age_str):
    if pd.isna(age_str):
        return None
    
    number, unit = age_str.split()
    number = float(number)
    
    if "year" in unit:
        return number
    elif "month" in unit:
        return number / 12
    elif "week" in unit:
        return number / 52
    elif "day" in unit:
        return number / 365
    else:
        return None  # in case of an unexpected format

# convert age to years
animal_test['Age'] = animal_test['Age upon Intake'].apply(age_to_years)
animal_test['Age'] = animal_test['Age'].fillna(0)  # fill NaN with 0
animal_test.drop('Age upon Intake', axis=1, inplace=True)

pd.set_option('display.max_columns', None)
animal_test.sample(5) # sample some data after transformations

,Id,Intake Time,Found Location,Intake Type,Intake Condition,Breed,Color,Cat,Sterilized,Male,Age
24663,24664,12/16/14 16:32,16419 Fm 969 in Manor (TX),Stray,Normal,Boxer,Tan,False,False,False,2.000000
20786,20787,4/25/19 14:45,Austin (TX),Stray,Normal,Domestic Shorthair Mix,Brown Tabby,True,False,True,0.076923
7106,7107,10/9/23 10:50,2601 Zaragosa Street in Austin (TX),Stray,Normal,Domestic Shorthair,Brown Tabby,True,False,False,0.333333
26544,26545,3/3/17 17:04,S Ih 35 Frontage Rd And E William Cannon Dr in...,Stray,Normal,German Shepherd Mix,White,False,False,False,0.333333
18075,18076,3/7/21 9:00,Austin (TX),Public Assist,Normal,Pit Bull,Black/White,False,False,True,2.000000


3. **Feature Engineering**: We need to one hot encode the categorical features. However, there are categoricals that have many labels, some with very little data points. We can study the trend of their similarities to each other and group to reduce dimensionality.

Let's look at Intake Condition and see all the possible labels for that class.

In [4]:
# Count the occurrences of each intake condition label
intake_condition_counts = animal_data['Intake Condition'].value_counts()

# Print the counts for each intake condition
for condition, count in intake_condition_counts.items():
    print(f"{condition}: {count}")

Normal: 95010
Injured: 6394
Sick: 4295
Nursing: 2957
Neonatal: 1240
Aged: 373
Medical: 298
Other: 247
Pregnant: 111
Feral: 104
Med Attn: 48
Behavior: 42
Unknown: 12
Neurologic: 10
Med Urgent: 7
Parvo: 5
Space: 2
Agonal: 1
Congenital: 1


We see above that there are 19 different categorical values for the Intake Condition feature, we can merge some of these rarer classification together.

Let's explore the outcome percentages of each intake condition so we can group these conditions better and reduce the amount of labels we need to one hot encode (increases dimensionality).

In [5]:
# Iterate through each unique intake condition and calculate outcome percentages
intake_conditions = animal_data['Intake Condition'].unique()

for condition in intake_conditions:
    condition_data = animal_data[animal_data['Intake Condition'] == condition]
    total_count = len(condition_data)
    if total_count > 0:
        outcome_percentages = condition_data['Outcome Type'].value_counts(normalize=True) * 100
        print(f"Intake Condition: {condition}")
        print(outcome_percentages)
        print("-" * 50)
    else:
        print(f"Intake Condition: {condition} has no entries.")
        print("-" * 50)

Intake Condition: Normal
Outcome Type
Adoption           52.596569
Transfer           29.360067
Return to Owner    15.985686
Euthanasia          1.513525
Died                0.544153
Name: proportion, dtype: float64
--------------------------------------------------
Intake Condition: Injured
Outcome Type
Adoption           34.813888
Transfer           30.669378
Euthanasia         18.720676
Return to Owner    12.605568
Died                3.190491
Name: proportion, dtype: float64
--------------------------------------------------
Intake Condition: Pregnant
Outcome Type
Transfer           54.954955
Adoption           39.639640
Return to Owner     4.504505
Died                0.900901
Name: proportion, dtype: float64
--------------------------------------------------
Intake Condition: Neonatal
Outcome Type
Transfer           69.919355
Adoption           25.564516
Died                3.064516
Return to Owner     0.967742
Euthanasia          0.483871
Name: proportion, dtype: float64
-------

Groupings based on similar outcome percentages

In [6]:
'''
   [Group]	       [Categories]
    Normal	        Normal
    Neonatal 	    Neonatal, Nursing
    Med_Minor	    Injured, Medical (more similar outcome adopt %)
    Med_Major       Med Attn, Med Urgent, Neurologic, Pregnant, Sick (more similar outcome transfer %)
    Behavioral	    Feral, Behavior
    Critical    	Agonal, Aged, Congenital, Parvo, Space, Other, Unknown
'''

'\n   [Group]\t       [Categories]\n    Normal\t        Normal\n    Neonatal \t    Neonatal, Nursing\n    Med_Minor\t    Injured, Medical (more similar outcome adopt %)\n    Med_Major       Med Attn, Med Urgent, Neurologic, Pregnant, Sick (more similar outcome transfer %)\n    Behavioral\t    Feral, Behavior\n    Critical    \tAgonal, Aged, Congenital, Parvo, Space, Other, Unknown\n'

We see that our merged class labels have relatively similar outcomes percentage-wise. This will reduce dimensionality and improve our training whilst not losing the originality of the groupings.

In [7]:
# # intake condition into grouped categories
# condition_map = {
#     'Normal': 'Normal',
#     'Injured': 'Med_Minor',
#     'Sick': 'Med_Major',
#     'Nursing': 'Neonatal',
#     'Neonatal': 'Neonatal',
#     'Med Attn': 'Med_Major',
#     'Med Urgent': 'Med_Major',
#     'Medical': 'Med_Minor',
#     'Neurologic': 'Med_Major',
#     'Pregnant': 'Med_Major',
#     'Feral': 'Behavioral',
#     'Behavior': 'Behavioral',
# }

# # map with a fallback to 'Rare'
# animal_data['Condition'] = animal_data['Intake Condition'].map(condition_map).fillna('Rare')
animal_data = pd.get_dummies(animal_data, columns=['Intake Condition'])
# animal_data.drop('Intake Condition', axis=1, inplace=True)

# animal_test['Condition'] = animal_test['Intake Condition'].map(condition_map).fillna('Rare')
animal_test = pd.get_dummies(animal_test, columns=['Intake Condition'])
# animal_test.drop('Intake Condition', axis=1, inplace=True)

animal_data.sample(5)  # sample some data after transformation

,Intake Time,Found Location,Intake Type,Breed,Color,Outcome Type,Cat,Sterilized,Male,Age,Intake Condition_Aged,Intake Condition_Agonal,Intake Condition_Behavior,Intake Condition_Congenital,Intake Condition_Feral,Intake Condition_Injured,Intake Condition_Med Attn,Intake Condition_Med Urgent,Intake Condition_Medical,Intake Condition_Neonatal,Intake Condition_Neurologic,Intake Condition_Normal,Intake Condition_Nursing,Intake Condition_Other,Intake Condition_Parvo,Intake Condition_Pregnant,Intake Condition_Sick,Intake Condition_Space,Intake Condition_Unknown
84582,06/30/2018 05:25:00 PM,Pocmont in Austin (TX),Stray,Jack Russell Terrier Mix,White/Brown,Adoption,False,False,True,2.000000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
22242,06/03/2017 01:41:00 PM,Austin (TX),Owner Surrender,Queensland Heeler Mix,Red Tick,Adoption,False,True,False,1.000000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
28251,08/03/2021 12:05:00 PM,1500 Fugles Grove in Manor (TX),Stray,Siberian Husky,Black/Tricolor,Adoption,False,False,True,0.416667,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
79895,07/30/2023 05:31:00 PM,S 1St St And W Oltorf St in Austin (TX),Stray,Labrador Retriever Mix,Brown Brindle,Adoption,False,False,False,0.750000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
47726,09/15/2018 03:05:00 PM,Austin (TX),Owner Surrender,Boxer Mix,Tan/White,Adoption,False,False,False,0.166667,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


Let's do the same for Intake Type and check the outcome percentages of each condition.

In [8]:
# observe outcome percentages for each intake type so we can group these categoricals, reduces dimensionality
intake_types = animal_data['Intake Type'].unique()

for intake_type in intake_types:
    type_data = animal_data[animal_data['Intake Type'] == intake_type]
    total_count = len(type_data)
    if total_count > 0:
        outcome_percentages = type_data['Outcome Type'].value_counts(normalize=True) * 100
        print(f"Intake Type: {intake_type}")
        print(outcome_percentages)
        print("-" * 50)
    else:
        print(f"Intake Type: {intake_type} has no entries.")
        print("-" * 50)

Intake Type: Stray
Outcome Type
Adoption           47.774546
Transfer           34.219783
Return to Owner    13.905286
Euthanasia          3.056120
Died                1.044266
Name: proportion, dtype: float64
--------------------------------------------------
Intake Type: Public Assist
Outcome Type
Return to Owner    63.545914
Adoption           18.528788
Transfer           14.222802
Euthanasia          3.262111
Died                0.440385
Name: proportion, dtype: float64
--------------------------------------------------
Intake Type: Owner Surrender
Outcome Type
Adoption           64.671050
Transfer           26.502541
Return to Owner     5.399357
Euthanasia          2.736980
Died                0.690073
Name: proportion, dtype: float64
--------------------------------------------------
Intake Type: Abandoned
Outcome Type
Adoption           63.017032
Transfer           26.845093
Return to Owner     9.083536
Euthanasia          0.648824
Died                0.405515
Name: proportion, 

These would be the groupings that are most similar to each other based on the outcome percentages.

In [9]:
'''
    [Group]	         [Categories]
     Public Assist    Public Assist
     Stray            Stray, Wildlife
     Owner-Initiated  Abandoned, Owner Surrender
     Euthanasia       Euthanasia Request
'''

'\n    [Group]\t         [Categories]\n     Public Assist    Public Assist\n     Stray            Stray, Wildlife\n     Owner-Initiated  Abandoned, Owner Surrender\n     Euthanasia       Euthanasia Request\n'

In [10]:
# # Define the mapping for grouping intake types
# intake_type_map = {
#     'Stray': 'Stray',
#     'Public Assist': 'Public Assist',
#     'Wildlife': 'Stray',
#     'Abandoned': 'Owner Initiated',
#     'Owner Surrender': 'Owner Initiated',
#     'Euthanasia Request': 'Euthanasia'
# }

# # Map the intake types to their respective groups
# animal_data['Intake'] = animal_data['Intake Type'].map(intake_type_map).fillna('Other')
animal_data = pd.get_dummies(animal_data, columns=['Intake Type'])
# animal_data.drop('Intake Type', axis=1, inplace=True)

# animal_test['Intake'] = animal_test['Intake Type'].map(intake_type_map).fillna('Other')
animal_test = pd.get_dummies(animal_test, columns=['Intake Type'])
# animal_test.drop('Intake Type', axis=1, inplace=True)

animal_data.sample(5)  # sample some data after transformation


,Intake Time,Found Location,Breed,Color,Outcome Type,Cat,Sterilized,Male,Age,Intake Condition_Aged,Intake Condition_Agonal,Intake Condition_Behavior,Intake Condition_Congenital,Intake Condition_Feral,Intake Condition_Injured,Intake Condition_Med Attn,Intake Condition_Med Urgent,Intake Condition_Medical,Intake Condition_Neonatal,Intake Condition_Neurologic,Intake Condition_Normal,Intake Condition_Nursing,Intake Condition_Other,Intake Condition_Parvo,Intake Condition_Pregnant,Intake Condition_Sick,Intake Condition_Space,Intake Condition_Unknown,Intake Type_Abandoned,Intake Type_Euthanasia Request,Intake Type_Owner Surrender,Intake Type_Public Assist,Intake Type_Stray,Intake Type_Wildlife
6612,06/21/2024 12:59:00 PM,South First Street And West Dittmar Road in Au...,Domestic Shorthair,Brown Tabby,Transfer,True,False,True,0.057692,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
103340,09/04/2014 02:55:00 PM,8801 N 620 #1333 in Austin (TX),Catahoula Mix,White/Black,Return to Owner,False,True,True,2.000000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
2482,09/21/2021 05:45:00 PM,8127 Mesa Drive in Austin (TX),Domestic Shorthair,Orange,Transfer,True,False,True,0.005479,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
85643,08/29/2015 01:12:00 PM,Austin (TX),Labrador Retriever Mix,Black,Adoption,False,True,True,9.000000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False
25237,10/04/2015 11:37:00 AM,1922 Wickersham Ln in Austin (TX),Domestic Shorthair Mix,Black,Adoption,True,False,False,0.083333,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False


Still need to find a way to encode:
- Intake Time
- Location Found
- Breed
- Color

How do they even affect the outcome or is there even a correlation? Are some colors / breeds more or less desirable than others (therefore changing adoption rates)?

In [11]:

# Filter data for breeds with "mix"
mix_breeds = animal_data[animal_data['Breed'].str.contains('mix', case=False, na=False)]

# Filter data for breeds without "mix"
non_mix_breeds = animal_data[~animal_data['Breed'].str.contains('mix', case=False, na=False)]

# Calculate percentages of Outcome Type for breeds with "mix"
mix_outcome_percentages = mix_breeds['Outcome Type'].value_counts(normalize=True) * 100

# Calculate percentages of Outcome Type for breeds without "mix"
non_mix_outcome_percentages = non_mix_breeds['Outcome Type'].value_counts(normalize=True) * 100

# Top 20 breeds with "mix" and their frequencies
top_mix_breeds = mix_breeds['Breed'].value_counts().head(20)

# Top 20 breeds without "mix" and their frequencies
top_non_mix_breeds = non_mix_breeds['Breed'].value_counts().head(20)

# Print the results
print("Top 20 breeds with 'mix' and their frequencies:")
print(top_mix_breeds)

print("\nTop 20 breeds without 'mix' and their frequencies:")
print(top_non_mix_breeds)

# Print the results
print("Outcome Type percentages for breeds with 'mix':")
print(mix_outcome_percentages)
print("\nOutcome Type percentages for breeds without 'mix':")
print(non_mix_outcome_percentages)

# Get the top 3 mixed breeds
top_3_mixed_breeds = mix_breeds['Breed'].value_counts().head(3).index

# Get the top 3 purebred breeds
top_3_purebred_breeds = non_mix_breeds['Breed'].value_counts().head(3).index

# Calculate and print outcome percentages for the top 3 mixed breeds
print("Outcome percentages for top 3 mixed breeds:")
for breed in top_3_mixed_breeds:
    breed_data = mix_breeds[mix_breeds['Breed'] == breed]
    outcome_percentages = breed_data['Outcome Type'].value_counts(normalize=True) * 100
    print(f"\nBreed: {breed}")
    print(outcome_percentages)

# Calculate and print outcome percentages for the top 3 purebred breeds
print("\nOutcome percentages for top 3 purebred breeds:")
for breed in top_3_purebred_breeds:
    breed_data = non_mix_breeds[non_mix_breeds['Breed'] == breed]
    outcome_percentages = breed_data['Outcome Type'].value_counts(normalize=True) * 100
    print(f"\nBreed: {breed}")
    print(outcome_percentages)

Top 20 breeds with 'mix' and their frequencies:
Breed
Domestic Shorthair Mix       25361
Pit Bull Mix                  6042
Labrador Retriever Mix        5654
Chihuahua Shorthair Mix       4896
German Shepherd Mix           2637
Domestic Medium Hair Mix      2564
Australian Cattle Dog Mix     1337
Domestic Longhair Mix         1254
Siamese Mix                   1106
Dachshund Mix                  852
Border Collie Mix              769
Boxer Mix                      749
Miniature Poodle Mix           646
Siberian Husky Mix             613
Australian Shepherd Mix        612
Catahoula Mix                  544
Yorkshire Terrier Mix          523
Great Pyrenees Mix             515
Miniature Schnauzer Mix        478
Rat Terrier Mix                475
Name: count, dtype: int64

Top 20 breeds without 'mix' and their frequencies:
Breed
Domestic Shorthair                    16046
Pit Bull                               2117
Domestic Medium Hair                   1436
Chihuahua Shorthair           

Let's try taking these four features out, from studying the data at a surface level, they don't seem to have too much relevancy to the final outcome.

Let's try building a KNN classifier with the remaining features. We can try techniques we have learned to find a good k-value.

In [12]:
animal_data.drop('Intake Time', axis=1, inplace=True)
animal_data.drop('Found Location', axis=1, inplace=True)
animal_data.drop('Breed', axis=1, inplace=True)
animal_data.drop('Color', axis=1, inplace=True)

animal_test.drop('Intake Time', axis=1, inplace=True)
animal_test.drop('Found Location', axis=1, inplace=True)
animal_test.drop('Breed', axis=1, inplace=True)
animal_test.drop('Color', axis=1, inplace=True)

animal_data.sample(5)

,Outcome Type,Cat,Sterilized,Male,Age,Intake Condition_Aged,Intake Condition_Agonal,Intake Condition_Behavior,Intake Condition_Congenital,Intake Condition_Feral,Intake Condition_Injured,Intake Condition_Med Attn,Intake Condition_Med Urgent,Intake Condition_Medical,Intake Condition_Neonatal,Intake Condition_Neurologic,Intake Condition_Normal,Intake Condition_Nursing,Intake Condition_Other,Intake Condition_Parvo,Intake Condition_Pregnant,Intake Condition_Sick,Intake Condition_Space,Intake Condition_Unknown,Intake Type_Abandoned,Intake Type_Euthanasia Request,Intake Type_Owner Surrender,Intake Type_Public Assist,Intake Type_Stray,Intake Type_Wildlife
5026,Transfer,False,False,False,0.416667,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
22236,Adoption,False,True,True,0.166667,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False
38651,Transfer,False,False,False,0.038462,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False
71839,Adoption,False,False,False,0.038462,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
64292,Transfer,False,True,False,8.000000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False


In [13]:
# Print the type of each feature in the training set
print("Feature types in the training set:")
print(animal_data.dtypes)

# Print the type of each feature in the test set
print("Feature types in the test set:")
print(animal_test.dtypes)

Feature types in the training set:
Outcome Type                       object
Cat                                  bool
Sterilized                           bool
Male                                 bool
Age                               float64
Intake Condition_Aged                bool
Intake Condition_Agonal              bool
Intake Condition_Behavior            bool
Intake Condition_Congenital          bool
Intake Condition_Feral               bool
Intake Condition_Injured             bool
Intake Condition_Med Attn            bool
Intake Condition_Med Urgent          bool
Intake Condition_Medical             bool
Intake Condition_Neonatal            bool
Intake Condition_Neurologic          bool
Intake Condition_Normal              bool
Intake Condition_Nursing             bool
Intake Condition_Other               bool
Intake Condition_Parvo               bool
Intake Condition_Pregnant            bool
Intake Condition_Sick                bool
Intake Condition_Space               bool

In [14]:
# Print the frequency of each Outcome Type
outcome_frequencies = animal_data['Outcome Type'].value_counts() / len(animal_data) * 100
print("Frequency of each Outcome Type:")
print(outcome_frequencies)

Frequency of each Outcome Type:
Outcome Type
Adoption           49.519149
Transfer           31.508587
Return to Owner    14.932933
Euthanasia          3.102819
Died                0.936513
Name: count, dtype: float64


Let's try training a histogram-based binning to build an ensemble of decision trees. The scikit-learn library has HistGradientBoostingClassifier which we can use to implement our approach. By using an ensemble of methods, we can more effectively balance predictions and work through class imbalances.

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter

# === 1. Load your dataset (replace this with actual loading) ===
# animal_data = pd.read_csv("animal_data.csv")  # or load however needed

# === 2. Separate labels and features ===
y = animal_data["Outcome Type"]
X = animal_data.drop(columns=["Outcome Type"])
X = X.fillna(0)

# === 3. Encode class labels ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# === 4. Train/Test split ===
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# === 5. Train classifier ===
# Assume you have label-encoded classes
class_counts = Counter(y_encoded)
total = sum(class_counts.values())

class_freqs = {cls: count / total for cls, count in class_counts.items()}
class_to_index = {label: i for i, label in enumerate(label_encoder.classes_)}

# we can play with these class weights to better balance the model and reduce the effects of class imbalance
class_weights = {
    class_to_index['Died']: 5.0,
    class_to_index['Euthanasia']: 4.0,
    class_to_index['Return to Owner']: 1.5,
    class_to_index['Transfer']: 1.0,
    class_to_index['Adoption']: 0.8
}

sample_weight = np.array([class_weights[label] for label in y_encoded])

clf = HistGradientBoostingClassifier(
    max_iter=200,
    max_depth=15,
    random_state=42
)

clf.fit(X, y_encoded, sample_weight=sample_weight)

# === 6. Predict and evaluate ===
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.6052986685858223
                 precision    recall  f1-score   support

       Adoption       0.66      0.74      0.70     10981
           Died       0.24      0.10      0.15       201
     Euthanasia       0.27      0.52      0.35       731
Return to Owner       0.51      0.64      0.57      3309
       Transfer       0.66      0.40      0.50      7010

       accuracy                           0.61     22232
      macro avg       0.47      0.48      0.45     22232
   weighted avg       0.62      0.61      0.60     22232



Hyperparam tuning:

In [16]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import LabelEncoder
# import pandas as pd
# import numpy as np

# # === 1. Prepare training data ===
# y = animal_data["Outcome Type"]
# X = animal_data.drop(columns=["Outcome Type"])
# X = X.replace([np.inf, -np.inf], np.nan).fillna(0)

# # === 2. Encode class labels ===
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)

# # === 3. Define parameter grid
# param_grid = {
#     'n_estimators': [50, 100, 150, 200],
#     'max_depth': [5, 10, 15, 20],
#     'min_samples_split': [2, 3, 4, 5],
#     'min_samples_leaf': [1, 2, 3, 4, 5]
# }

# # === 4. Setup grid search with 5-fold CV
# grid_search = GridSearchCV(
#     estimator=RandomForestClassifier(random_state=42, class_weight='balanced'),
#     param_grid=param_grid,
#     cv=5,
#     scoring='accuracy',
#     n_jobs=-1,
#     verbose=2
# )

# # === 5. Run the grid search
# grid_search.fit(X, y_encoded)

# # === 6. View best params and score
# print("Best Parameters:", grid_search.best_params_)
# print("Best Cross-Validated Accuracy:", grid_search.best_score_)


Now let's build the entire model using the training set and test on the test set, also outputs the final predictions to csv file.

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from collections import Counter

# === 1. Encode categorical features and prepare training data ===
X = pd.get_dummies(animal_data.drop(columns=["Outcome Type"]), dtype=np.uint8)
y = animal_data["Outcome Type"]

# Save the column structure of training features
feature_columns = X.columns

# === 2. Prepare test data and reindex to match training columns ===
X_test_final = pd.get_dummies(animal_test.drop(columns=["Id"]), dtype=np.uint8)
X_test_final = X_test_final.reindex(columns=feature_columns, fill_value=0)

# === 3. Encode class labels ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# === 4. Class weights and mappings ===
class_counts = Counter(y_encoded)
total = sum(class_counts.values())
class_to_index = {label: i for i, label in enumerate(label_encoder.classes_)}
class_weights = {
    class_to_index['Died']: 5.0,
    class_to_index['Euthanasia']: 4.0,
    class_to_index['Return to Owner']: 1.5,
    class_to_index['Transfer']: 1.0,
    class_to_index['Adoption']: 0.8
}
sample_weight = np.array([class_weights[label] for label in y_encoded])

# === 5. Train classifier ===
clf = HistGradientBoostingClassifier(
    max_iter=200,
    max_depth=15,
    random_state=42
)
clf.fit(X, y_encoded, sample_weight=sample_weight)

# === 6. Predict on test set ===
y_test_pred_encoded = clf.predict(X_test_final)
y_test_pred = label_encoder.inverse_transform(y_test_pred_encoded)

# === 7. Save predictions to CSV ===
submission = pd.DataFrame({
    "Id": animal_test["Id"],
    "Outcome Type": y_test_pred
})
submission = submission.sort_values("Id").reset_index(drop=True)
submission.to_csv("animal_test_predictions.csv", index=False)
print("✅ Saved predictions to 'animal_test_predictions.csv'")

# === 8. Show prediction distribution as percentages ===
prediction_distribution = submission["Outcome Type"].value_counts(normalize=True) * 100
print("\nPrediction Distribution (%):")
print(prediction_distribution.round(2).sort_index().to_string())


✅ Saved predictions to 'animal_test_predictions.csv'

Prediction Distribution (%):
Outcome Type
Adoption           55.25
Died                0.37
Euthanasia          6.64
Return to Owner    18.20
Transfer           19.55


Work in Progress: Potentially train cats and dogs separately?

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter

# === 1. Prepare features and labels ===
y = animal_data["Outcome Type"]
X = animal_data.drop(columns=["Outcome Type"])
X = X.fillna(0)

# === 2. Encode labels ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# === 3. Split by species
cat_mask = X["Cat"] == True
dog_mask = X["Cat"] == False

X_cat = X[cat_mask]
y_cat = y_encoded[cat_mask]

X_dog = X[dog_mask]
y_dog = y_encoded[dog_mask]

# === 4. Train/Test split for both sets
X_cat_train, X_cat_test, y_cat_train, y_cat_test = train_test_split(X_cat, y_cat, test_size=0.2, random_state=42)
X_dog_train, X_dog_test, y_dog_train, y_dog_test = train_test_split(X_dog, y_dog, test_size=0.2, random_state=42)

# === 5. Define class weights (same logic for both models)
class_to_index = {label: i for i, label in enumerate(label_encoder.classes_)}
class_weights = {
    class_to_index['Died']: 5.0,
    class_to_index['Euthanasia']: 4.0,
    class_to_index['Return to Owner']: 1.5,
    class_to_index['Transfer']: 1.0,
    class_to_index['Adoption']: 0.8
}

# === 6. Create sample weights
sample_weight_cat = np.array([class_weights[label] for label in y_cat_train])
sample_weight_dog = np.array([class_weights[label] for label in y_dog_train])

# === 7. Train models
clf_cat = HistGradientBoostingClassifier(max_iter=200, max_depth=15, random_state=42)
clf_dog = HistGradientBoostingClassifier(max_iter=200, max_depth=15, random_state=42)

clf_cat.fit(X_cat_train, y_cat_train, sample_weight=sample_weight_cat)
clf_dog.fit(X_dog_train, y_dog_train, sample_weight=sample_weight_dog)

# === 8. Predict & Evaluate
y_cat_pred = clf_cat.predict(X_cat_test)
y_dog_pred = clf_dog.predict(X_dog_test)

print("=== 🐱 Cat Model Evaluation ===")
print("Accuracy:", accuracy_score(y_cat_test, y_cat_pred))
print(classification_report(y_cat_test, y_cat_pred, target_names=label_encoder.classes_))

print("=== 🐶 Dog Model Evaluation ===")
print("Accuracy:", accuracy_score(y_dog_test, y_dog_pred))
print(classification_report(y_dog_test, y_dog_pred, target_names=label_encoder.classes_))


=== 🐱 Cat Model Evaluation ===
Accuracy: 0.6308758537565288
                 precision    recall  f1-score   support

       Adoption       0.71      0.69      0.70      4922
           Died       0.07      0.04      0.05       124
     Euthanasia       0.27      0.61      0.38       399
Return to Owner       0.38      0.43      0.41       440
       Transfer       0.66      0.60      0.63      4071

       accuracy                           0.63      9956
      macro avg       0.42      0.47      0.43      9956
   weighted avg       0.65      0.63      0.64      9956

=== 🐶 Dog Model Evaluation ===
Accuracy: 0.5863473444118605
                 precision    recall  f1-score   support

       Adoption       0.62      0.79      0.69      5977
           Died       0.09      0.05      0.06        65
     Euthanasia       0.22      0.29      0.25       294
Return to Owner       0.58      0.66      0.62      2974
       Transfer       0.53      0.14      0.23      2966

       accuracy     

Feature cleaning and engineering take #2, we can try predicting on the model we had earlier, it didn't perform too poorly but class imbalance was an obvious issue.

In [19]:
animal_data.sample(5) # sample some data after transformations

,Outcome Type,Cat,Sterilized,Male,Age,Intake Condition_Aged,Intake Condition_Agonal,Intake Condition_Behavior,Intake Condition_Congenital,Intake Condition_Feral,Intake Condition_Injured,Intake Condition_Med Attn,Intake Condition_Med Urgent,Intake Condition_Medical,Intake Condition_Neonatal,Intake Condition_Neurologic,Intake Condition_Normal,Intake Condition_Nursing,Intake Condition_Other,Intake Condition_Parvo,Intake Condition_Pregnant,Intake Condition_Sick,Intake Condition_Space,Intake Condition_Unknown,Intake Type_Abandoned,Intake Type_Euthanasia Request,Intake Type_Owner Surrender,Intake Type_Public Assist,Intake Type_Stray,Intake Type_Wildlife
105861,Adoption,True,False,False,0.416667,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False
14142,Adoption,False,True,False,10.000000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False
14070,Adoption,False,False,False,1.000000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
102835,Adoption,False,False,True,3.000000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
99505,Died,True,False,False,0.083333,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False


In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter

# === 1. Separate labels and features ===
y = animal_data["Outcome Type"]
X = animal_data.drop(columns=["Outcome Type"])
X = X.fillna(0)

# === 2. Encode class labels ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# === 3. Train/Test split ===
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# === 4. Train model without sample weights ===
clf = HistGradientBoostingClassifier(
    max_iter=200,
    max_depth=15,
    random_state=42
)

clf.fit(X_train, y_train)

# === 5. Predict and evaluate ===
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.6178481468154012
                 precision    recall  f1-score   support

       Adoption       0.63      0.80      0.71     10981
           Died       0.20      0.00      0.01       201
     Euthanasia       0.43      0.17      0.25       731
Return to Owner       0.59      0.54      0.57      3309
       Transfer       0.62      0.42      0.50      7010

       accuracy                           0.62     22232
      macro avg       0.49      0.39      0.41     22232
   weighted avg       0.61      0.62      0.60     22232



In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier

# === 1. Encode categorical features and prepare training data ===
X = pd.get_dummies(animal_data.drop(columns=["Outcome Type"]), dtype=np.uint8)
y = animal_data["Outcome Type"]

# Save the column structure of training features
feature_columns = X.columns

# === 2. Prepare test data and reindex to match training columns ===
X_test_final = pd.get_dummies(animal_test.drop(columns=["Id"]), dtype=np.uint8)
X_test_final = X_test_final.reindex(columns=feature_columns, fill_value=0)

# === 3. Encode class labels ===
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# === 4. Train classifier (no class weights) ===
clf = HistGradientBoostingClassifier(
    max_iter=200,
    max_depth=15,
    random_state=42
)
clf.fit(X, y_encoded)

# === 5. Predict on test set ===
y_test_pred_encoded = clf.predict(X_test_final)
y_test_pred = label_encoder.inverse_transform(y_test_pred_encoded)

# === 6. Save predictions to CSV ===
submission = pd.DataFrame({
    "Id": animal_test["Id"],
    "Outcome Type": y_test_pred
})
submission = submission.sort_values("Id").reset_index(drop=True)
submission.to_csv("animal_test_predictions.csv", index=False)
print("✅ Saved predictions to 'animal_test_predictions.csv'")

# === 7. Show prediction distribution as percentages ===
prediction_distribution = submission["Outcome Type"].value_counts(normalize=True) * 100
print("\nPrediction Distribution (%):")
print(prediction_distribution.round(2).sort_index().to_string())


✅ Saved predictions to 'animal_test_predictions.csv'

Prediction Distribution (%):
Outcome Type
Adoption           63.96
Died                0.03
Euthanasia          1.32
Return to Owner    13.45
Transfer           21.24


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

# === 1. Prepare data ===
y = animal_data["Outcome Type"]
X = animal_data.drop(columns=["Outcome Type"]).fillna(0)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
class_labels = label_encoder.classes_
class_to_index = {label: i for i, label in enumerate(class_labels)}

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# === 2. Train multiclass model ===
clf_multi = HistGradientBoostingClassifier(max_iter=200, max_depth=15, random_state=42)
clf_multi.fit(X_train, y_train)
probs_multi = clf_multi.predict_proba(X_test)

# === 3. Train OvR classifiers ===
def train_ovr(target_label):
    y_binary = (y_train == class_to_index[target_label]).astype(int)
    clf = HistGradientBoostingClassifier(max_iter=200, max_depth=15, random_state=42)
    clf.fit(X_train, y_binary)
    return clf

clf_died = train_ovr("Died")
clf_euth = train_ovr("Euthanasia")

# === 4. Get OvR probabilities ===
prob_died = clf_died.predict_proba(X_test)[:, 1]
prob_euth = clf_euth.predict_proba(X_test)[:, 1]

# === 5. Define threshold search function ===
def find_best_ovr_thresholds(probs_multi, prob_died, prob_euth, y_true, class_to_index, class_labels):
    thresholds = np.arange(0.01, 0.51, 0.01)
    best_thresholds = {}
    best_f1 = 0
    best_combo = (0.15, 0.18)

    for t_died in thresholds:
        for t_euth in thresholds:
            y_pred = np.argmax(probs_multi, axis=1)
            for i in range(len(y_pred)):
                if prob_died[i] > t_died:
                    y_pred[i] = class_to_index["Died"]
                elif prob_euth[i] > t_euth:
                    y_pred[i] = class_to_index["Euthanasia"]
            score = f1_score(y_true, y_pred, average='macro')
            if score > best_f1:
                best_f1 = score
                best_combo = (t_died, t_euth)

    return {
        "best_thresholds": {
            "Died": round(best_combo[0], 2),
            "Euthanasia": round(best_combo[1], 2)
        },
        "best_macro_f1": round(best_f1, 4)
    }

# === 6. Search for optimal thresholds ===
result = find_best_ovr_thresholds(
    probs_multi=probs_multi,
    prob_died=prob_died,
    prob_euth=prob_euth,
    y_true=y_test,
    class_to_index=class_to_index,
    class_labels=class_labels
)

print("Best Thresholds:")
print(result["best_thresholds"])
print("Best Macro F1 Score:")
print(result["best_macro_f1"])

# === 7. Apply optimal thresholds to predictions ===
optimal_died_thresh = result["best_thresholds"]["Died"]
optimal_euth_thresh = result["best_thresholds"]["Euthanasia"]

y_pred_final = np.argmax(probs_multi, axis=1)
for i in range(len(y_pred_final)):
    if prob_died[i] > optimal_died_thresh:
        y_pred_final[i] = class_to_index["Died"]
    elif prob_euth[i] > optimal_euth_thresh:
        y_pred_final[i] = class_to_index["Euthanasia"]

# === 8. Final evaluation ===
print("\n=== Final Evaluation with Optimized Thresholds ===")
print("Accuracy:", accuracy_score(y_test, y_pred_final))
print(classification_report(y_test, y_pred_final, target_names=class_labels))


Best Thresholds:
{'Died': np.float64(0.07), 'Euthanasia': np.float64(0.18)}
Best Macro F1 Score:
0.4422

=== Final Evaluation with Optimized Thresholds ===
Accuracy: 0.6112360561353005
                 precision    recall  f1-score   support

       Adoption       0.63      0.79      0.70     10981
           Died       0.11      0.12      0.11       201
     Euthanasia       0.32      0.36      0.34       731
Return to Owner       0.59      0.54      0.56      3309
       Transfer       0.64      0.40      0.49      7010

       accuracy                           0.61     22232
      macro avg       0.46      0.44      0.44     22232
   weighted avg       0.61      0.61      0.60     22232



In [23]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

# 1. Label encode main target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(animal_data["Outcome Type"])
class_to_index = {label: i for i, label in enumerate(label_encoder.classes_)}

# 2. One-hot encode data
X = pd.get_dummies(animal_data.drop(columns=["Outcome Type"]), dtype=np.uint8)
X_test_final = pd.get_dummies(animal_test.drop(columns=["Id"]), dtype=np.uint8)
X_test_final = X_test_final.reindex(columns=X.columns, fill_value=0)

# 3. Train multiclass model (no class weights)
clf_multi = HistGradientBoostingClassifier(max_iter=200, max_depth=15, random_state=42)
clf_multi.fit(X, y_encoded)
probs_multi = clf_multi.predict_proba(X_test_final)

# 4. Train OvR binary classifiers
def train_ovr_classifier(class_name):
    y_binary = (animal_data["Outcome Type"] == class_name).astype(int)
    clf = HistGradientBoostingClassifier(max_iter=200, max_depth=15, random_state=42)
    clf.fit(X, y_binary)
    return clf

clf_died = train_ovr_classifier("Died")
clf_euth = train_ovr_classifier("Euthanasia")

# 5. Predict probabilities from OvR
prob_died = clf_died.predict_proba(X_test_final)[:, 1]
prob_euth = clf_euth.predict_proba(X_test_final)[:, 1]

# 6. Multiclass prediction (initial)
y_test_pred_encoded = np.argmax(probs_multi, axis=1)
y_test_pred = label_encoder.inverse_transform(y_test_pred_encoded)

# 7. Override multiclass with OvR if confident enough
thresholds = {
    "Died": 0.07,
    "Euthanasia": 0.18
}
for i in range(len(y_test_pred)):
    if prob_died[i] > thresholds["Died"]:
        y_test_pred[i] = "Died"
    elif prob_euth[i] > thresholds["Euthanasia"]:
        y_test_pred[i] = "Euthanasia"

# 8. Save to CSV
submission = pd.DataFrame({
    "Id": animal_test["Id"],
    "Outcome Type": y_test_pred
})
submission = submission.sort_values("Id").reset_index(drop=True)
submission.to_csv("animal_test_predictions_with_ovr.csv", index=False)
print("✅ Saved combined OvR + multiclass predictions to 'animal_test_predictions_with_ovr.csv'")

# 9. Show distribution
distribution = submission["Outcome Type"].value_counts(normalize=True) * 100
print("\nPrediction Distribution (%):")
print(distribution.sort_index().round(2).to_string())


✅ Saved combined OvR + multiclass predictions to 'animal_test_predictions_with_ovr.csv'

Prediction Distribution (%):
Outcome Type
Adoption           62.67
Died                0.81
Euthanasia          3.63
Return to Owner    13.28
Transfer           19.61
